<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Atividade4Corrigida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import dlib
import numpy as np
import pandas as pd
from deepface import DeepFace
import mediapipe as mp
from fpdf import FPDF

# Configuração do Mediapipe para detecção de atividades
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detectar_faces(frame, detector):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return frame, len(faces)

def analisar_emocao(frame):
    try:
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return analysis[0]['dominant_emotion']
    except:
        return "desconhecido"

def detectar_atividade(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    return "Movimento detectado" if results.pose_landmarks else "Sem atividade"

def processar_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
    detector = dlib.get_frontal_face_detector()
    dados = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame, num_faces = detectar_faces(frame, detector)
        emocao = analisar_emocao(frame)
        atividade = detectar_atividade(frame)
        dados.append([frame_count, num_faces, emocao, atividade])
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return dados

def gerar_relatorio(dados, relatorio_path):
    df = pd.DataFrame(dados, columns=['Frame', 'Rostos', 'Expressão', 'Atividade'])
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "Relatório de Análise de Vídeo", ln=True, align='C')
    pdf.cell(200, 10, f"Total de Frames: {len(df)}", ln=True, align='L')
    pdf.cell(200, 10, f"Rostos Detectados: {df['Rostos'].sum()}", ln=True, align='L')
    pdf.output(relatorio_path)

def main():
    video_path = "data/input_video.mp4"
    output_path = "results/output_video.avi"
    relatorio_path = "results/relatorio.pdf"
    dados = processar_video(video_path, output_path)
    gerar_relatorio(dados, relatorio_path)
    print("Processamento concluído! Relatório gerado.")

# Executar o pipeline
main()
